In [114]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import re
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import json
import datasets
import transformers
from transformers import AutoTokenizer, default_data_collator, get_scheduler
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining
from torch.utils.data import DataLoader
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForSequenceClassification, BertModel
from torch.optim import AdamW

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

import sys
import math

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


In [2]:
#WARNING
#cacheディレクトリの指定は絶対マウント先のフォルダにするように
#そうしないとdockerイメージを管理している研究室サーバーのルートディレクトリ(/)がパンパンになってしまう
#/が容量オーバーすると何も動かなくなって他の人に迷惑
!export HF_DATASETS_CACHE="/home/"

In [3]:
df_downsampled = pd.read_csv("./logdata/downsampled_anomaly.csv", usecols=["BlockId", "Label", "log"])

In [4]:
bert_model

,log,BlockId,Label
0,081109 203521 145 INFO dfs.DataNode$DataXceive...,blk_-3544583377289625738,Anomaly
1,081109 203521 29 INFO dfs.FSNamesystem: BLOCK*...,blk_-3544583377289625738,Anomaly
2,081109 203522 144 INFO dfs.DataNode$DataXceive...,blk_-3544583377289625738,Anomaly
3,081109 203522 145 INFO dfs.DataNode$DataXceive...,blk_-3544583377289625738,Anomaly
4,081109 203522 147 INFO dfs.DataNode$PacketResp...,blk_-3544583377289625738,Anomaly
...,...,...,...
576495,081111 030649 32 INFO dfs.FSNamesystem: BLOCK*...,blk_4241722502979736601,Normal
576496,081110 010143 30 INFO dfs.FSNamesystem: BLOCK*...,blk_2863437573282244260,Normal
576497,081111 040435 33 INFO dfs.FSNamesystem: BLOCK*...,blk_-8441598874045640693,Normal
576498,081111 044332 29 INFO dfs.FSNamesystem: BLOCK*...,blk_795775988001366663,Normal


In [178]:
df_summary = pd.read_csv("./logdata/log_summary_pairs.csv", usecols=["input", "summary"])
df_summary

,input,summary
0,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
1,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
2,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
3,INFO dfs.DataNode$DataXceiver : Receiving bloc...,Receiving block src; blockMap updated; Verific...
4,INFO dfs.DataNode$PacketResponder : PacketResp...,Receiving block src; blockMap updated; Verific...
...,...,...
11975,INFO storage.BlockManager : Removing RDD 12744,Removing RDD;
11976,INFO storage.BlockManager : Removing RDD 12740,Removing RDD;
11977,INFO storage.BlockManager : Removing RDD 12736,Removing RDD;
11978,INFO storage.BlockManager : Removing RDD 12732,Removing RDD;


In [5]:
# def log2list(path):
#     with open(path, "r") as f:
#         lines = [line.rstrip("\n") for line in f]
#     return lines

# hdfs = log2list("./logdata/hdfs/HDFS.log")
# log_label_pairs = {}

# df_hdfs_logs = pd.DataFrame({"log" : hdfs})
# df_hdfs_logs["BlockId"] = df_hdfs_logs["log"].map(lambda line : re.search(r"blk_[-]*[0-9]+", line).group(0))
# df_hdfs = df_hdfs_anomaly.merge(df_hdfs_logs, how="left", on=["BlockId"])

# df_hdfs_without_duplicate = df_hdfs.drop_duplicates(["BlockId"])
# df_hdfs_without_duplicate.to_csv("./logdata/hdfs/df_hdfs_without_duplicate.csv")

In [6]:
# df_hdfs_without_duplicate["text"] = df_hdfs_without_duplicate["log"].map(preprocess_log)

In [7]:
import transformers
transformers.__version__

'4.31.0.dev0'

In [16]:
#分類と事前学習で異なるトークナイザを利用している？
#根拠: 例えばHDFSの異常検知の場合, 異常ラベルはログデータのブロック番号と紐づけられている

In [181]:
#NOTE
#ここで記号を区切ってwordninjaで塊除去を行っている
#なお, かなり時間がかかるので実行には注意が必要
#もう少し賢く書きたかったが, その書き方を思いつく時間とナイーブに実行する時間はどうせ同じぐらいだろう
def preprocess_log(text, remove_digits):
    text = text.replace('[', " ")
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace(":", " ")
    text = text.replace("/", " ")
    text = text.replace(";", " ")
    text = text.replace("=", " ")
    text = text.replace("*", " ")
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    text = text.lower()
    text = " ".join(wordninja.split(text))
    remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
    if remove_digits:
        return remove_num(text)
    return text;

def preprocess_log_batch_hof(remove_digits=True):
    def preprocess_log_batch(example):
        return {
            "text": [preprocess_log(l, remove_digits=remove_digits) for l in example["log"]]
        }
    return preprocess_log_batch

In [9]:
# df_ad_hdfs = pd.read_csv("./logdata/hdfs/hdfs_log_label_pairs.csv", usecols=["log", "BlockId", "Label"])
# df_hdfs_without_duplicate["text"] = df_hdfs_without_duplicate["log"].map(preprocess_log)

In [10]:
# ダウンサンプリングした
# df_ad_hdfs["Label"].value_counts(normalize=True)
# df_equal = df_ad_hdfs[df_ad_hdfs["Label"]=="Anomaly"]
# df_normal_sampled = df_ad_hdfs[df_ad_hdfs["Label"]=="Normal"].sample(len(df_equal))
# df_equal = pd.concat([df_equal, df_normal_sampled])
# df_equal.to_csv("./logdata/downsampled_anomaly.csv")
# df_equal

In [4]:
#NOTE
#Loading train test dataset used in loglizer here for the sake of fair comparison

TEST_DATA_FILE_PATH = "./logdata/hdfs/loglizer_hdfs_test_data.csv"
TRAIN_DATA_FILE_PATH = "./logdata/hdfs/df_hdfs_without_duplicate.csv"


dataset_hdfs_test = load_dataset("csv", data_files=TEST_DATA_FILE_PATH)
dataset_hdfs_test = dataset_hdfs_test.remove_columns(['Unnamed: 0'])

dataset_hdfs_train = load_dataset("csv", data_files=TRAIN_DATA_FILE_PATH)
dataset_hdfs_train = dataset_hdfs_train.remove_columns(['Unnamed: 0'])

train_valid = dataset_hdfs_train["train"].train_test_split(0.2)
train_data = train_valid["train"]
eval_dataset = train_valid["test"]

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-a438c5b2cd2acdf4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 589.83it/s]
Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-8d2c1362a4f07327/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 111.88it/s]


In [6]:
train_data.to_pandas()["Label"].value_counts(normalize=True)

Normal     0.970714
Anomaly    0.029286
Name: Label, dtype: float64

In [11]:
dataset_hdfs_train

DatasetDict({
    train: Dataset({
        features: ['BlockId', 'Label', 'log'],
        num_rows: 575061
    })
})

In [14]:
df_train = pd.read_csv(TRAIN_DATA_FILE_PATH, usecols=["BlockId", "Label", "log"])

In [15]:
#NOTE
#Removing all test data here

df_test = pd.read_csv(TEST_DATA_FILE_PATH, usecols=["log", "BlockId", "Label"])
test_blk_ids = list(df_test["BlockId"])
df_clean = df_train[~df_train["BlockId"].isin(test_blk_ids)]
df_clean.to_csv("./logdata/hdfs/downsampled_anomaly_without_test.csv")

In [16]:
# Loading larger dataset that does not contain test data used in loglizer 
datased_ad_hdfs = load_dataset("csv", data_files="./logdata/hdfs/anomaly_without_test.csv")
datased_ad_hdfs = datased_ad_hdfs["train"].train_test_split(0.2)
datased_ad_hdfs = datased_ad_hdfs.remove_columns(['Unnamed: 0'])

train_data = datased_ad_hdfs["train"]
eval_dataset = datased_ad_hdfs["test"]

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-2a9c33563e8a79ae/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 118.78it/s]


In [19]:
train_data = train_data.map(
    preprocess_log_batch_hof(remove_digits=False),
    batched=True,
)

In [20]:
eval_dataset = eval_dataset.map(
    preprocess_log_batch_hof(remove_digits=False),
    batched=True,
)

In [21]:
log_tokenizer = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_large/")
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/")

batch_size=16  # change to 16 for full training

def process_data_to_model_inputs_from_target(target_tokenizer, target_tag="log"):
    def process_data_to_model_inputs(batch):
        # tokenize the inputs and labels
        # NOTE 
        # x != "Normal"の符号を==にしてしまっていたのがミスの原因
        batch["labels"] = list(map(lambda x : int(x != "Normal"), batch["Label"]))

        return target_tokenizer(batch[target_tag], padding="max_length", truncation=True, max_length=180)
    return process_data_to_model_inputs



In [22]:
tokenized_train_data = train_data.map(
    process_data_to_model_inputs_from_target(log_tokenizer, target_tag="text"), 
    batched=True, 
    batch_size=10000, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)
tokenized_train_data

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 456872
})

In [27]:
train_data[0]["text"]

'081110 223339 15898 info df s data node data x ce iver receiving block b lk 1029163488700913789 s rc 10 250 10 176 59011 de st 10 250 10 176 50010'

In [26]:
log_tokenizer.decode(tokenized_train_data[0]["input_ids"])

'[CLS] 081110 223339 15898 info df s data node data x ce iver receiving block b lk 1029163488700913789 s rc 10 250 10 176 59011 de st 10 250 10 176 50010 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [52]:
tokenized_train_data.set_format(type="torch")

In [53]:
# small_train_dataset = tokenized_train_data.shuffle(seed=42).select(range(1000))


In [28]:
eval_dataset = eval_dataset.map(
    process_data_to_model_inputs_from_target(log_tokenizer, target_tag="text"), 
    batched=True, 
    batch_size=10000, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)

eval_dataset.set_format(type="torch")
# small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(1000))

In [151]:
dataset_hdfs_test = load_dataset("csv", data_files=TEST_DATA_FILE_PATH)
dataset_hdfs_test = dataset_hdfs_test.remove_columns(['Unnamed: 0'])
test_dataset = dataset_hdfs_test.map(
    preprocess_log_batch_hof(remove_digits=True),
    batched=True,
)["train"]

tokenized_test_dataset = test_dataset.map(
    process_data_to_model_inputs_from_target(log_tokenizer_w_n, target_tag="text"),
    batched=True, 
    batch_size=batch_size, 
    remove_columns=['log', 'BlockId', 'Label', 'text']
)


tokenized_test_dataset.set_format(type="torch")
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=8)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-a438c5b2cd2acdf4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)

100%|██████████| 1/1 [00:00<00:00, 654.64it/s]

Map: 100%|██████████| 3971/3971 [00:04<00:00, 961.43 examples/s]
                                                                
Map:  89%|████████▉ | 3552/3971 [00:00<00:00, 4392.74 examples/s]
                                                                 

In [72]:
df_test = test_dataset.to_pandas()

In [77]:
df_test_anomaly = df_test[df_test.Label!="Normal"]
df_test_normal = df_test[df_test.Label=="Normal"]

In [91]:
df_test_normal.iloc[963]["log"]

'081109 204352 33 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /user/root/rand/_temporary/_task_200811092030_0001_m_000345_0/part-00345. blk_-8975646672319015268'

In [92]:
df_test_anomaly["log"].iloc[0]

'081109 204346 29 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /user/root/rand/_temporary/_task_200811092030_0001_m_000240_0/part-00240. blk_8522440494057928612'

In [99]:
len(df_test_anomaly["text"])

157

In [228]:
#fasttextでログをベクトル化
def vectorize_log(target_model):
    def vectorize_log_helper(input_log):
        v = np.zeros(100, dtype=float)

        for w in input_log.split():
            v += target_model[w] 
        return v / len(input_log.split())
    return vectorize_log_helper

#ref: https://necromuralist.github.io/Neurotic-Networking/posts/nlp/machine-translation-k-nearest-neighbors/index.html

def cosine_similarity(vector_1: np.ndarray, vector_2: np.ndarray) -> float:
    """Calculates the similarity between two vectors

    Args:
     vector_1: array to compare
     vector_2: array to compare to vector_1

    Returns:
     cosine similarity between the two vectors
    """
    return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1) *
                                          np.linalg.norm(vector_2))

#fasttextでベクトル化したログデータのうち,k近傍のベクトルを得ている
#なお, 論文(https://arxiv.org/abs/2305.15778)ではユークリッド距離と時系列を考慮しているが, 単純化のためここでは単なるコサイン類似度を利用
def nearest_neighbor(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    # cosine_similarities = [cosine_similarity(v, row) for row in candidates]

    # for each candidate vector...
    #for row in candidates:
    #    # get the cosine similarity
    #    cos_similarity = cosine_similarity(v, row)
    #
    #    # append the similarity to the list
    #    similarity_l.append(cos_similarity)

    # sort the similarity list and get the indices of the sorted list
    # sorted_ids = numpy.argsort(similarity_l)

    # get the indices of the k most similar candidate vectors
    # k_idx = sorted_ids[-k:]
    ### END CODE HERE ###
    return np.argsort([cosine_similarity(v, row) for row in candidates])[-k:]

def show_current_log(idx):
    for column in df_summary.columns:
        if column != "log_vector":
            print(f"{column}: ", df_summary.iloc[idx, :][column])

In [177]:
output["pooler_output"].squeeze().numpy().shape

(128,)

In [93]:
df_test_normal["text"].iloc[0]

'081109 204208 26 info df s fs name system block name system allocate block user root rand temporary task 200811092030 0001 m 000043 0 part 00043 b lk 942782950815104487'

In [33]:
log_tokenizer.decode(tokenized_test_dataset[100]["input_ids"])

'[CLS] 081109 204217 33 info df s fs name system block name system allocate block user root rand temporary task 200811092030 0001 m 000390 0 part 00390 b lk 4575784335805628598 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

# ここまで

In [34]:
test_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

0    0.960463
1    0.039537
Name: labels, dtype: float64

In [35]:
train_dataloader = DataLoader(tokenized_train_data, shuffle=True, batch_size=32)
eval_dataloader = DataLoader(eval_dataset, batch_size=32)

In [36]:
train_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

0    0.970782
1    0.029218
Name: labels, dtype: float64

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [250]:
dataset_summary

Dataset({
    features: ['log', 'summary'],
    num_rows: 11980
})

In [38]:


unilogConfigmodel = BertForSequenceClassification.from_pretrained("./logdata/unilog_pretrain_with_numbers_12_layers_2/", num_labels=2).to(device)
optimizer = AdamW(unilogConfigmodel.parameters(), lr=5e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./logdata/unilog_pretrain_with_numbers_12_layers_2/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [67]:
num_training_steps // 3

14278

In [42]:
import evaluate

In [54]:
SAVE_NAME = "./logdata/log_anomaly_with_numbers_no_downsampling"

In [ ]:
train_steps = 0

save_steps = 1000

best_f1 = 0.0

for epoch in range(num_epochs):
    unilogConfigmodel.train()
    pbar = tqdm(train_dataloader)
    for batch in train_dataloader:
        train_steps += 1
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = unilogConfigmodel(**batch)
        loss = outputs.loss
        loss.backward()
        pbar.update(1)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if train_steps % save_steps == 0:
            pbar.write(f"Loss:{loss}")
            #TODO 
            unilogConfigmodel.save_pretrained(f"{SAVE_NAME}_{train_steps}")
    import evaluate

    metric_acc = evaluate.load("accuracy")
    metric_f1 = evaluate.load("f1")
    metric_precision = evaluate.load("precision")
    metric_recall = evaluate.load("recall")

    unilogConfigmodel.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = unilogConfigmodel(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric_acc.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1.add_batch(predictions=predictions, references=batch["labels"])
        metric_precision.add_batch(predictions=predictions, references=batch["labels"])
        metric_recall.add_batch(predictions=predictions, references=batch["labels"])

    pbar.write(str(metric_acc.compute()))
    pbar.write(str(metric_precision.compute()))
    pbar.write(str(metric_recall.compute()))
    
    curr_f1_score = metric_f1.compute()["f1"]
    prev_f1_score = best_f1
    pbar.write(f"F1: {str(curr_f1_score)}")
    if (curr_f1_score > prev_f1_score):
        best_f1 = max(best_f1, curr_f1_score)
        unilogConfigmodel.save_pretrained(f"./{SAVE_NAME}_best")


train_dataloader.dataset

In [62]:
test_dataloader.dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3971
})

In [68]:
import evaluate

metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")

unilogBestModel = BertForSequenceClassification.from_pretrained("./logdata/log_anomaly_with_numbers_no_downsampling_15000/", num_labels=2).to(device)
unilogBestModel.eval()

gt = []
prediction_list = []

for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = unilogBestModel(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    gt += list(batch["labels"].cpu().numpy())
    prediction_list += list(predictions.cpu().numpy())
    
    metric_acc.add_batch(predictions=predictions, references=batch["labels"])
    metric_f1.add_batch(predictions=predictions, references=batch["labels"])
    metric_precision.add_batch(predictions=predictions, references=batch["labels"])
    metric_recall.add_batch(predictions=predictions, references=batch["labels"])
    
print(metric_acc.compute())
print(metric_f1.compute())
print(metric_precision.compute())
print(metric_recall.compute())


100%|██████████| 497/497 [00:06<00:00, 75.18it/s]

{'accuracy': 0.9604633593553261}
{'f1': 0.0}
{'precision': 0.0}
{'recall': 0.0}



/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
test_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

0    0.960463
1    0.039537
Name: labels, dtype: float64

In [69]:
test_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

0    0.960463
1    0.039537
Name: labels, dtype: float64

In [37]:
test_dataloader.dataset.to_pandas()["labels"].value_counts(normalize=True)

1    0.960463
0    0.039537
Name: labels, dtype: float64

In [19]:
print("""{'accuracy': 0.9604633593553261}
{'f1': 0.9798330122029544}
{'precision': 0.9604633593553261}
{'recall': 1.0}""")

{'accuracy': 0.9604633593553261}
{'f1': 0.9798330122029544}
{'precision': 0.9604633593553261}
{'recall': 1.0}
